In [1]:
import os
import sys
import math
import logging
from pathlib import Path

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from enadepy.loaders import read_raw
from enadepy.index import get_index_dict

In [3]:
df = read_raw('../data/raw/microdados_enade_2018.csv', low_memory=False)

In [4]:
df.shape

(548127, 137)

In [5]:
df.head()

,NU_ANO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_CURSO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO,NU_IDADE,TP_SEXO,ANO_FIM_EM,ANO_IN_GRAD,CO_TURNO_GRADUACAO,TP_INSCRICAO_ADM,TP_INSCRICAO,NU_ITEM_OFG,NU_ITEM_OFG_Z,NU_ITEM_OFG_X,NU_ITEM_OFG_N,NU_ITEM_OCE,NU_ITEM_OCE_Z,NU_ITEM_OCE_X,NU_ITEM_OCE_N,DS_VT_GAB_OFG_ORIG,DS_VT_GAB_OFG_FIN,DS_VT_GAB_OCE_ORIG,DS_VT_GAB_OCE_FIN,DS_VT_ESC_OFG,DS_VT_ACE_OFG,DS_VT_ESC_OCE,DS_VT_ACE_OCE,TP_PRES,TP_PR_GER,TP_PR_OB_FG,TP_PR_DI_FG,TP_PR_OB_CE,TP_PR_DI_CE,TP_SFG_D1,TP_SFG_D2,TP_SCE_D1,TP_SCE_D2,TP_SCE_D3,NT_GER,NT_FG,NT_OBJ_FG,NT_DIS_FG,NT_FG_D1,NT_FG_D1_PT,NT_FG_D1_CT,NT_FG_D2,NT_FG_D2_PT,NT_FG_D2_CT,NT_CE,NT_OBJ_CE,NT_DIS_CE,NT_CE_D1,NT_CE_D2,NT_CE_D3,...,QE_I09,QE_I10,QE_I11,QE_I12,QE_I13,QE_I14,QE_I15,QE_I16,QE_I17,QE_I18,QE_I19,QE_I20,QE_I21,QE_I22,QE_I23,QE_I24,QE_I25,QE_I26,QE_I27,QE_I28,QE_I29,QE_I30,QE_I31,QE_I32,QE_I33,QE_I34,QE_I35,QE_I36,QE_I37,QE_I38,QE_I39,QE_I40,QE_I41,QE_I42,QE_I43,QE_I44,QE_I45,QE_I46,QE_I47,QE_I48,QE_I49,QE_I50,QE_I51,QE_I52,QE_I53,QE_I54,QE_I55,QE_I56,QE_I57,QE_I58,QE_I59,QE_I60,QE_I61,QE_I62,QE_I63,QE_I64,QE_I65,QE_I66,QE_I67,QE_I68
0,2018,1356,10003,10020,1,47116,1,3546603,35,3,22,M,2013,2015,4,0,0,8,0,0,0,27,1,8,0,CACBEBDE,CACBEBDE,ADBBDEDCEEDCAEZADBCBDABECBB,XXBXDEDXEEDCAEZADBCBXAXECXX,CACCDDDD,11100010,BADBDADAEACAABAADBECBDACCCA,990910191000108111009090199,555,555,555,555,555,555,555,555,333,333,555,42.8,50.6,50.0,51.5,63.0,75.0,60.0,40.0,60.0,35.0,40.2,44.4,16.7,0.0,0.0,50.0,...,B,A,H,A,D,A,A,50,A,A,B,I,A,B,B,A,C,<NA>,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,5
1,2018,1356,10003,10020,1,47116,1,3546603,35,3,49,F,1988,2015,4,0,0,8,0,0,0,27,1,8,0,CACBEBDE,CACBEBDE,ADBBDEDCEEDCAEZADBCBDABECBB,XXBXDEDXEEDCAEZADBCBXAXECXX,CAADEDDE,11001011,BBDBEEDAEBCEAEBADBCBBEAECCA,990901191000118111119091199,555,555,555,555,555,555,555,555,555,555,555,59.3,57.9,62.5,51.0,55.0,75.0,50.0,47.0,55.0,45.0,59.7,66.7,20.0,40.0,20.0,0.0,...,E,E,H,A,D,F,A,35,A,C,C,H,A,C,C,A,A,<NA>,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6
2,2018,1356,10003,10020,1,47116,1,3546603,35,3,23,M,2013,2015,4,0,0,8,0,0,0,27,1,8,0,CACBEBDE,CACBEBDE,ADBBDEDCEEDCAEZADBCBDABECBB,XXBXDEDXEEDCAEZADBCBXAXECXX,CAAACBBE,11000101,CDDBDCDAEEDCACDADDCBBDAECCA,990910191111108110119091199,555,555,555,555,555,555,555,555,555,555,555,60.9,47.2,50.0,43.0,33.0,65.0,25.0,53.0,65.0,50.0,65.4,72.2,26.7,40.0,40.0,0.0,...,D,E,H,A,B,A,A,35,A,A,B,A,B,D,D,B,C,<NA>,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,5,6,6,6,6,5,6,6,6,5,6,6,6,6,6,6,6,6,6,6,5
3,2018,1356,10003,10020,1,47116,1,3546603,35,3,25,M,2011,2015,4,0,0,8,0,0,0,27,1,8,0,CACBEBDE,CACBEBDE,ADBBDEDCEEDCAEZADBCBDABECBB,XXBXDEDXEEDCAEZADBCBXAXECXX,BAADDBCC,01000100,BACAAADBEBEACDDBCACBAABACBC,990900191000008000119190199,555,555,555,555,555,555,336,333,333,555,333,25.0,15.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.3,33.3,0.0,0.0,0.0,0.0,...,D,A,H,A,B,A,A,35,A,A,B,A,A,E,E,A,A,C,6,6,6,6,6,6,6,6,6,6,6,5,6,6,6,6,6,6,6,6,6,6,6,6,5,6,6,6,6,6,6,6,6,6,6,6,5,6,6,6,6,4
4,2018,1356,10003,10020,1,47116,1,3546603,35,3,22,F,2014,2015,4,0,0,8,0,0,0,27,1,8,0,CACBEBDE,CACBEBDE,ADBBDEDCEEDCAEZADBCBDABECBB,XXBXDEDXEEDCAEZADBCBXAXECXX,CABBDBBE,11010101,BEDEDCDBBADADEDACEECAEAECCA,990910190010018100009091199,555,555,555,555,555,555,555,555,555,555,555,42.4,58.1,62.5,51.5,42.0,50.0,40.0,61.0,65.0,60.0,37.1,38.9,26.7,40.0,40.0,0.0,...,D,A,H,A,A,A,A,35,A,A,B,H,B,B,B,A,A,<NA>,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6


In [6]:
df2 = read_raw('../data/raw/microdados_enade_2017.csv', low_memory=False)

In [7]:
df2.shape

(537436, 150)

In [8]:
df2.head()

,NU_ANO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_CURSO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO,NU_IDADE,TP_SEXO,ANO_FIM_EM,ANO_IN_GRAD,CO_TURNO_GRADUACAO,TP_INSCRICAO_ADM,TP_INSCRICAO,NU_ITEM_OFG,NU_ITEM_OFG_Z,NU_ITEM_OFG_X,NU_ITEM_OFG_N,NU_ITEM_OCE,NU_ITEM_OCE_Z,NU_ITEM_OCE_X,NU_ITEM_OCE_N,DS_VT_GAB_OFG_ORIG,DS_VT_GAB_OFG_FIN,DS_VT_GAB_OCE_ORIG,DS_VT_GAB_OCE_FIN,DS_VT_ESC_OFG,DS_VT_ACE_OFG,DS_VT_ESC_OCE,DS_VT_ACE_OCE,TP_PRES,TP_PR_GER,TP_PR_OB_FG,TP_PR_DI_FG,TP_PR_OB_CE,TP_PR_DI_CE,TP_SFG_D1,TP_SFG_D2,TP_SCE_D1,TP_SCE_D2,TP_SCE_D3,NT_GER,NT_FG,NT_OBJ_FG,NT_DIS_FG,NT_FG_D1,NT_FG_D1_PT,NT_FG_D1_CT,NT_FG_D2,NT_FG_D2_PT,NT_FG_D2_CT,NT_CE,NT_OBJ_CE,NT_DIS_CE,NT_CE_D1,NT_CE_D2,NT_CE_D3,...,QE_I22,QE_I23,QE_I24,QE_I25,QE_I26,QE_I27,QE_I28,QE_I29,QE_I30,QE_I31,QE_I32,QE_I33,QE_I34,QE_I35,QE_I36,QE_I37,QE_I38,QE_I39,QE_I40,QE_I41,QE_I42,QE_I43,QE_I44,QE_I45,QE_I46,QE_I47,QE_I48,QE_I49,QE_I50,QE_I51,QE_I52,QE_I53,QE_I54,QE_I55,QE_I56,QE_I57,QE_I58,QE_I59,QE_I60,QE_I61,QE_I62,QE_I63,QE_I64,QE_I65,QE_I66,QE_I67,QE_I68,QE_I69,QE_I70,QE_I71,QE_I72,QE_I73,QE_I74,QE_I75,QE_I76,QE_I77,QE_I78,QE_I79,QE_I80,QE_I81
0,2017,1,1,10028,5710,3,1,5103403,51,5,26,F,2007,2012,3,0,1,8,0,0,0,27,0,8,0,CCBBCEAD,CCBBCEAD,EDCAEABCDBDDDCBCBBEADDCBEDB,EDXXEABCXBXDDCXCXBEADXCBEXB,DCBBCEAD,01111111,EEEADDBCABADCCAAAEEDDDCAACB,109900119191019090101910091,555,555,555,555,555,555,555,333,555,555,333,52.6,64.1,87.5,29.0,58.0,50.0,60.0,0.0,0.0,0.0,48.7,52.6,26.7,55.0,25.0,0.0,...,B,D,E,H,I,5,5,4,3,3,4,3,5,3,5,2,2,3,1,1,5,1,2,2,2,1,2,2,5,5,2,3,1,3,3,4,3,3,5,3,2,2,2,1,4,4,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2017,1,1,10028,5710,3,1,5103403,51,5,23,F,2013,2013,3,0,1,8,0,0,0,27,0,8,0,CCBBCEAD,CCBBCEAD,EDCAEABCDBDDDCBCBBEADDCBEDB,EDXXEABCXBXDDCXCXBEADXCBEXB,CCBBCEBD,11111101,ECDEAADCABADDCDEAADCDCCBECB,109901019191119090001911191,555,555,555,555,555,555,555,555,555,555,555,63.1,82.1,87.5,74.0,72.0,60.0,75.0,76.0,80.0,75.0,56.7,63.2,20.0,60.0,0.0,0.0,...,B,C,E,C,A,4,4,4,3,6,6,6,6,5,6,5,5,6,4,6,6,3,3,6,6,6,5,3,6,6,6,6,3,3,5,5,6,5,6,6,4,6,4,3,6,6,6,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2017,1,1,10028,5710,3,1,5103403,51,5,23,M,2011,2013,3,0,1,8,0,0,0,27,0,8,0,CCBBCEAD,CCBBCEAD,EDCAEABCDBDDDCBCBBEADDCBEDB,EDXXEABCXBXDDCXCXBEADXCBEXB,CCDBCEAD,11011111,EADEEAAACAAECDACDEEBCCECBEA,109911009090009190100900090,555,555,555,333,555,555,333,333,555,333,333,33.7,52.5,87.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.4,26.3,33.3,100.0,0.0,0.0,...,A,B,A,E,A,5,4,6,1,1,6,6,3,6,6,1,1,6,1,1,4,1,1,4,4,1,1,3,6,6,4,4,1,3,2,1,2,2,3,2,2,2,7,1,1,1,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2017,1,1,10028,5710,3,1,5103403,51,5,23,M,2011,2013,3,0,1,8,0,0,0,27,0,8,0,CCBBCEAD,CCBBCEAD,EDCAEABCDBDDDCBCBBEADDCBEDB,EDXXEABCXBXDDCXCXBEADXCBEXB,CABBCEBD,10111101,BEDDEABECBADDACCEACBDECAEAB,009911109191109190001910191,555,555,555,555,555,555,555,555,555,555,555,58.1,72.6,75.0,69.0,67.0,75.0,65.0,71.0,75.0,70.0,53.2,57.9,26.7,80.0,0.0,0.0,...,B,C,A,E,A,5,4,5,4,6,4,6,6,5,5,5,4,4,4,5,6,3,3,5,4,4,3,4,6,5,3,6,5,5,5,4,5,5,6,4,5,5,5,5,5,5,5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2017,1,1,10028,5710,3,1,5103403,51,5,24,M,2010,2013,3,0,1,8,0,0,0,27,0,8,0,CCBBCEAD,CCBBCEAD,EDCAEABCDBDDDCBCBBEADDCBEDB,EDXXEABCXBXDDCXCXBEADXCBEXB,CCBBCDAE,11111010,EDDDDDBEAABDDDAECBBBEEEDAAB,119900109091109091000900091,555,555,555,555,555,555,555,555,555,555,555,46.3,73.2,75.0,70.5,75.0,75.0,75.0,66.0,50.0,70.0,37.3,36.8,40.0,0.0,100.0,20.0,...,C,B,C,A,C,3,3,2,3,3,3,2,3,3,3,2,2,2,3,5,4,2,2,2,3,3,3,2,4,4,2,4,1,3,3,3,3,2,5,2,2,2,3,3,4,2,3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [9]:
df2[df2.QE_I69.notna()].head()

,NU_ANO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_CURSO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO,NU_IDADE,TP_SEXO,ANO_FIM_EM,ANO_IN_GRAD,CO_TURNO_GRADUACAO,TP_INSCRICAO_ADM,TP_INSCRICAO,NU_ITEM_OFG,NU_ITEM_OFG_Z,NU_ITEM_OFG_X,NU_ITEM_OFG_N,NU_ITEM_OCE,NU_ITEM_OCE_Z,NU_ITEM_OCE_X,NU_ITEM_OCE_N,DS_VT_GAB_OFG_ORIG,DS_VT_GAB_OFG_FIN,DS_VT_GAB_OCE_ORIG,DS_VT_GAB_OCE_FIN,DS_VT_ESC_OFG,DS_VT_ACE_OFG,DS_VT_ESC_OCE,DS_VT_ACE_OCE,TP_PRES,TP_PR_GER,TP_PR_OB_FG,TP_PR_DI_FG,TP_PR_OB_CE,TP_PR_DI_CE,TP_SFG_D1,TP_SFG_D2,TP_SCE_D1,TP_SCE_D2,TP_SCE_D3,NT_GER,NT_FG,NT_OBJ_FG,NT_DIS_FG,NT_FG_D1,NT_FG_D1_PT,NT_FG_D1_CT,NT_FG_D2,NT_FG_D2_PT,NT_FG_D2_CT,NT_CE,NT_OBJ_CE,NT_DIS_CE,NT_CE_D1,NT_CE_D2,NT_CE_D3,...,QE_I22,QE_I23,QE_I24,QE_I25,QE_I26,QE_I27,QE_I28,QE_I29,QE_I30,QE_I31,QE_I32,QE_I33,QE_I34,QE_I35,QE_I36,QE_I37,QE_I38,QE_I39,QE_I40,QE_I41,QE_I42,QE_I43,QE_I44,QE_I45,QE_I46,QE_I47,QE_I48,QE_I49,QE_I50,QE_I51,QE_I52,QE_I53,QE_I54,QE_I55,QE_I56,QE_I57,QE_I58,QE_I59,QE_I60,QE_I61,QE_I62,QE_I63,QE_I64,QE_I65,QE_I66,QE_I67,QE_I68,QE_I69,QE_I70,QE_I71,QE_I72,QE_I73,QE_I74,QE_I75,QE_I76,QE_I77,QE_I78,QE_I79,QE_I80,QE_I81
40,2017,1,1,10028,2001,6,1,5103403,51,5,31,F,2005,2008,2,0,1,8,0,1,0,27,0,4,0,CCBBCEAD,XCBBCEAD,EAADDACDEBBAACDDDBBCAEADCBB,EAADDAXDEBBAXXDDDBBCAEADCBX,<NA>,<NA>,<NA>,<NA>,222,222,222,222,222,222,222,222,222,222,222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,D,B,E,E,F,4,4,4,4,4,4,4,4,5,4,4,4,4,4,4,4,4,4,4,3,4,3,4,4,4,3,3,3,4,3,4,4,4,2,3,3,3,5,3,4,4,4,A,B,E,A,A,B,A,A,A,B,B,B,B
41,2017,1,1,10028,2001,6,1,5103403,51,5,37,F,2013,2014,2,0,1,8,0,1,0,27,0,4,0,CCBBCEAD,XCBBCEAD,EAADDACDEBBAACDDDBBCAEADCBB,EAADDAXDEBBAXXDDDBBCAEADCBX,CAEBBAAE,90010010,ABAAEEABBBDCEADDBCABEEACEBE,001000900100991100000110019,555,555,555,555,555,555,555,555,555,555,555,30.7,31.8,28.6,36.5,20.0,40.0,15.0,53.0,65.0,50.0,30.3,30.4,30.0,30.0,60.0,0.0,...,C,B,E,H,A,3,3,3,3,5,2,5,6,3,4,3,4,2,2,4,6,2,3,3,2,2,2,3,3,3,1,2,2,3,3,3,3,3,3,4,3,3,3,3,3,3,3,A,E,I,H,A,B,B,A,C,D,C,B,B
42,2017,1,1,10028,2001,6,1,5103403,51,5,59,F,1983,2014,2,0,1,8,0,1,0,27,0,4,0,CCBBCEAD,XCBBCEAD,EAADDACDEBBAACDDDBBCAEADCBB,EAADDAXDEBBAXXDDDBBCAEADCBX,BEDECEBE,90001100,EBDADCDEEDEDADCECDBDDAA*DDB,100010901000990000100010009,555,555,555,555,555,555,555,555,555,555,333,26.8,38.6,28.6,53.5,36.0,60.0,30.0,71.0,55.0,75.0,22.9,21.7,30.0,30.0,60.0,0.0,...,C,C,E,E,A,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,6,6,6,A,A,I,I,A,B,B,A,A,A,A,A,A
43,2017,1,1,10028,2001,6,1,5103403,51,5,26,F,2010,2014,1,0,1,8,0,1,0,27,0,4,0,CCBBCEAD,XCBBCEAD,EAADDACDEBBAACDDDBBCAEADCBB,EAADDAXDEBBAXXDDDBBCAEADCBX,DDABEEBB,90010100,CADADAADEBBAAADEDBDCEAABCBB,010011911111991011010010119,555,555,555,555,555,555,555,555,336,336,336,52.4,43.4,28.6,65.5,61.0,65.0,60.0,70.0,50.0,75.0,55.4,65.2,0.0,0.0,0.0,0.0,...,A,C,C,H,A,5,5,4,6,6,5,6,6,4,4,4,3,3,3,4,5,3,1,2,5,3,4,1,4,4,1,1,3,3,3,5,1,2,1,3,3,2,4,4,3,1,2,A,H,I,I,A,A,A,A,E,C,C,B,B
44,2017,1,1,10028,2001,6,1,5103403,51,5,52,F,1985,2014,2,0,1,8,0,1,0,27,0,4,0,CCBBCEAD,XCBBCEAD,EAADDACDEBBAACDDDBBCAEADCBB,EAADDAXDEBBAXXDDDBBCAEADCBX,DACECEDE,90001100,BBBADCCEECBDDBBDABDBBEBCECC,000010901010990101000100009,555,555,555,555,555,555,555,555,555,555,555,31.9,36.4,28.6,48.0,96.0,80.0,100.0,0.0,0.0,0.0,30.4,26.1,55.0,45.0,80.0,40.0,...,C,B,E,E,A,4,7,3,1,3,5,4,5,5,4,5,3,3,2,7,2,1,1,3,2,1,1,1,1,2,1,8,1,1,1,2,4,1,1,1,1,1,3,8,2,2,3,A,C,B,A,A,B,A,A,A,C,C,C,C


In [10]:
df3 = read_raw('../data/interim/microdados_enade_2016.csv', low_memory=False)

In [11]:
df3.head()

,NU_ANO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_CURSO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO,NU_IDADE,TP_SEXO,ANO_FIM_EM,ANO_IN_GRAD,NU_ITEM_OFG,NU_ITEM_OFG_Z,NU_ITEM_OFG_X,NU_ITEM_OFG_N,NU_ITEM_OCE,NU_ITEM_OCE_Z,NU_ITEM_OCE_X,NU_ITEM_OCE_N,DS_VT_GAB_OFG_ORIG,DS_VT_GAB_OFG_FIN,DS_VT_GAB_OCE_ORIG,DS_VT_GAB_OCE_FIN,DS_VT_ESC_OFG,DS_VT_ACE_OFG,DS_VT_ESC_OCE,DS_VT_ACE_OCE,TP_PRES,TP_PR_GER,TP_PR_OB_FG,TP_PR_DI_FG,TP_PR_OB_CE,TP_PR_DI_CE,TP_SFG_D1,TP_SFG_D2,TP_SCE_D1,TP_SCE_D2,TP_SCE_D3,NT_GER,NT_FG,NT_OBJ_FG,NT_DIS_FG,NT_FG_D1,NT_FG_D1_PT,NT_FG_D1_CT,NT_FG_D2,NT_FG_D2_PT,NT_FG_D2_CT,NT_CE,NT_OBJ_CE,NT_DIS_CE,NT_CE_D1,NT_CE_D2,NT_CE_D3,CO_RS_I1,CO_RS_I2,CO_RS_I3,...,QE_I12,QE_I13,QE_I14,QE_I15,QE_I16,QE_I17,QE_I18,QE_I19,QE_I20,QE_I21,QE_I22,QE_I23,QE_I24,QE_I25,QE_I26,QE_I27,QE_I28,QE_I29,QE_I30,QE_I31,QE_I32,QE_I33,QE_I34,QE_I35,QE_I36,QE_I37,QE_I38,QE_I39,QE_I40,QE_I41,QE_I42,QE_I43,QE_I44,QE_I45,QE_I46,QE_I47,QE_I48,QE_I49,QE_I50,QE_I51,QE_I52,QE_I53,QE_I54,QE_I55,QE_I56,QE_I57,QE_I58,QE_I59,QE_I60,QE_I61,QE_I62,QE_I63,QE_I64,QE_I65,QE_I66,QE_I67,QE_I68,TP_INSCRICAO,TP_INSCRICAO_ADM,CO_TURNO_GRADUACAO
0,2016,1,10002,10028,38,7,1,5103403,51,5,35,F,2009,2010,8,0,0,0,27,4,4,0,CAEBCDAC,CAEBCDAC,DCDEZBBCBZAECBZBACAEDZECCEA,DXXEZBBCBZAECBZBACAEDZECXEX,<NA>,<NA>,<NA>,<NA>,222,222,222,222,222,222,222,222,222,222,222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.,.,.,...,E,F,A,A,51,A,B,A,G,A,C,E,E,H,G,6,6,6,6,6,4,5,5,5,6,6,4,5,1,4,6,3,5,6,6,3,3,6,6,5,8,8,6,6,3,6,6,1,1,5,4,4,1,1,6,6,3,<NA>,<NA>,1
1,2016,1,10002,10028,38,7,1,5103403,51,5,24,F,2008,2013,8,0,0,0,27,4,4,0,CAEBCDAC,CAEBCDAC,DCDEZBBCBZAECBZBACAEDZECCEA,DXXEZBBCBZAECBZBACAEDZECXEX,CABADDBD,11000100,EEEEDBBABAAEEBABAAAEDEECDCB,099181101811018110111811909,555,555,555,333,555,555,333,333,555,333,555,52.6,22.5,37.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,62.6,73.7,0.0,0.0,0.0,0.0,D,D,A,...,E,A,A,A,51,A,A,A,H,B,D,C,A,E,A,4,4,3,4,4,4,5,6,4,4,4,5,5,4,5,4,4,5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,<NA>,<NA>,1
2,2016,1,10002,10028,38,7,1,5103403,51,5,21,F,2012,2013,8,0,0,0,27,4,4,0,CAEBCDAC,CAEBCDAC,DCDEZBBCBZAECBZBACAEDZECCEA,DXXEZBBCBZAECBZBACAEDZECXEX,CBEBADAC,10110111,DABEDBBCBDAECEDBACAEDEECEEB,199181111811108111111811919,555,555,555,555,555,555,555,555,555,555,555,83.6,69.0,75.0,60.0,55.0,75.0,50.0,65.0,85.0,60.0,88.5,94.7,53.3,40.0,80.0,40.0,C,C,B,...,A,C,A,A,35,D,A,D,B,B,D,C,A,H,I,5,5,4,3,6,5,6,6,6,5,4,6,6,2,5,5,4,5,5,6,4,4,6,6,6,1,1,4,5,5,6,4,4,1,5,7,7,5,4,7,5,5,<NA>,<NA>,3
3,2016,1,10002,10028,38,7,1,5103403,51,5,30,F,2004,2012,8,0,0,0,27,4,4,0,CAEBCDAC,CAEBCDAC,DCDEZBBCBZAECBZBACAEDZECCEA,DXXEZBBCBZAECBZBACAEDZECXEX,AEEBADAE,00110110,EDDEEBBCDEABEDDBEABEEDECDBB,099181110810008100010811909,555,555,555,555,555,555,555,555,555,555,333,43.6,44.4,50.0,36.0,13.0,65.0,0.0,59.0,55.0,60.0,43.3,47.4,20.0,60.0,0.0,0.0,C,D,A,...,A,A,A,A,51,B,A,B,K,A,C,B,E,E,I,6,6,5,6,6,6,6,6,6,6,5,5,6,4,5,6,4,4,6,4,6,5,6,6,6,4,4,6,6,3,6,6,4,4,4,3,5,4,4,6,6,4,<NA>,<NA>,3
4,2016,1,10002,10028,38,7,1,5103403,51,5,23,F,2009,2013,8,0,0,0,27,4,4,0,CAEBCDAC,CAEBCDAC,DCDEZBBCBZAECBZBACAEDZECCEA,DXXEZBBCBZAECBZBACAEDZECXEX,CAEACDBC,11101101,ACBCDBBCBEAECDDBADAEDDECAED,099081111811108110111811919,555,555,555,555,555,555,555,555,555,555,555,72.8,65.8,75.0,52.0,68.0,60.0,70.0,36.0,60.0,30.0,75.1,78.9,53.3,40.0,60.0,60.0,C,D,C,...,F,B,A,E,50,A,A,B,C,A,E,C,A,A,A,6,6,6,6,6,6,6,6,6,6,6,6,6,4,6,6,6,6,6,5,6,6,6,6,6,6,6,4,6,6,6,6,6,2,4,5,5,3,2,6,4,5,<NA>,<NA>,3


In [12]:
print(df3.dtypes.apply(lambda x: x.name))

NU_ANO                int64
CO_IES                int64
CO_CATEGAD            int64
CO_ORGACAD            int64
CO_GRUPO              int64
                      ...  
QE_I67                Int64
QE_I68                Int64
TP_INSCRICAO          Int64
TP_INSCRICAO_ADM      Int64
CO_TURNO_GRADUACAO    Int64
Length: 137, dtype: object


## Dataset summary

### Enade 2018

In [13]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NU_ANO,548127.0,2.018000e+03,0.000000,2018.0,2018.0,2018.0,2018.0,2018.0
CO_IES,548127.0,1.505268e+03,2531.483186,1.0,322.0,670.0,1561.0,23410.0
CO_CATEGAD,548127.0,7.959837e+03,4030.074430,93.0,10002.0,10005.0,10007.0,17634.0
CO_ORGACAD,548127.0,1.002439e+04,3.530985,10019.0,10022.0,10022.0,10028.0,10028.0
CO_GRUPO,548127.0,6.666748e+01,176.484641,1.0,2.0,13.0,38.0,804.0
CO_CURSO,548127.0,3.306784e+05,689785.415918,1.0,20762.0,77713.0,116140.0,5001389.0
CO_MODALIDADE,548127.0,1.176833e+00,0.381528,1.0,1.0,1.0,1.0,2.0
CO_MUNIC_CURSO,548127.0,3.465790e+06,870451.436376,1100023.0,3106200.0,3530607.0,4113205.0,5300108.0
CO_UF_CURSO,548127.0,3.447284e+01,8.702386,11.0,31.0,35.0,41.0,53.0
CO_REGIAO_CURSO,548127.0,3.107015e+00,0.962333,1.0,3.0,3.0,4.0,5.0


### Enade 2017

In [14]:
df2.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NU_ANO,537436.0,2.017000e+03,0.000000,2017.0,2017.0,2017.0,2017.0,2017.0
CO_IES,537436.0,1.385499e+03,2504.935866,1.0,322.0,583.0,1472.0,19739.0
CO_CATEGAD,537436.0,3.345377e+00,1.567353,1.0,2.0,4.0,5.0,7.0
CO_ORGACAD,537436.0,1.002523e+04,3.484124,10019.0,10022.0,10028.0,10028.0,10028.0
CO_GRUPO,537436.0,3.192640e+03,2044.616513,21.0,2001.0,2402.0,5710.0,6409.0
CO_CURSO,537436.0,3.748719e+05,685490.838248,3.0,34476.0,97507.0,311303.0,5001385.0
CO_MODALIDADE,537436.0,7.867393e-01,0.409611,0.0,1.0,1.0,1.0,1.0
CO_MUNIC_CURSO,537436.0,3.401669e+06,887485.584488,1100023.0,3106200.0,3509502.0,4106902.0,5300108.0
CO_UF_CURSO,537436.0,3.383609e+01,8.867267,11.0,31.0,35.0,41.0,53.0
CO_REGIAO_CURSO,537436.0,3.042857e+00,0.981309,1.0,3.0,3.0,4.0,5.0


### Enade 2016

In [15]:
df3.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
NU_ANO,216044,2016,0,2016,2016,2016,2016,2016
CO_IES,216044,1544.17,2440.37,1,344,621,1753,19512
CO_CATEGAD,216044,8379.79,3666.83,93,10002,10005,10008,17634
CO_ORGACAD,216044,10024.6,3.44936,10019,10022,10026,10028,10028
CO_GRUPO,216044,400.409,1071.17,5,19,28,38,3501
CO_CURSO,216044,371714,831991,7,37979,91897,119916,5.00118e+06
CO_MODALIDADE,216044,0.903612,0.295123,0,1,1,1,1
CO_MUNIC_CURSO,216044,3.35167e+06,951383,1.10002e+06,2.7043e+06,3.30611e+06,4.1069e+06,5.30011e+06
CO_UF_CURSO,216044,33.3497,9.50293,11,27,33,41,53
CO_REGIAO_CURSO,216044,2.988,1.04319,1,2,3,4,5
